In [1]:
pip install torch torchvision torchaudio transformers pillow

In [2]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import cv2

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
sample_num = 5
image_dict = {}
sample_dict = {}

for i in range(sample_num):
    name = "OCR_Test_" + str(i+1) + ".png"
    image = cv2.imread(name)
    image_dict[str(i)] = image
    # processing, if there is a better way of corping then replace this
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (image.shape[1]//2, 1))
    dilated = cv2.dilate(th, kernel, iterations=2)

    cnts, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    line_boxes = [cv2.boundingRect(c) for c in cnts]
    line_boxes = sorted(line_boxes, key=lambda x: x[1])

    sample_dict[str(i)] = line_boxes

In [12]:
def trocr(img):
    pixel_values = processor(images=img, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    return processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

for key, item in image_dict.items():
    image = item
    line_boxes = sample_dict[key]
    results = []
    for (x,y,w,h) in line_boxes:
        line_img = image[y:y+h, x:x+w]
        pil = Image.fromarray(cv2.cvtColor(line_img, cv2.COLOR_BGR2RGB))
        results.append(trocr(pil))

    print("\n".join(results))

ACCEPT AT9% (REGUST) MUST
ABOG. JORGE BREADYICH MUSA
1.00
LETIZIA PAREDES ARIAS (JUEZ/A)
BALANA SUMMARYLE
TRIBINAL DE APELACION 2A. SALA PENAI
OF. EUGENIO TIMENEE R.


In [13]:
#without splitting image(so only single line)
for key, item in image_dict.items():
    pil = Image.fromarray(cv2.cvtColor(item, cv2.COLOR_BGR2RGB))
    print(trocr(pil))

ACCOST REG:8000000-
ABOG. AORUARIO JUDICI CH MUSA
ITEMATA BARELMETS ARIAS (DUEZ/A)
WWW.COM.
OF. EUGENIO TIMENEE R.


In [9]:
# TODO: integrate this loss function with corping pipeline
def levenshtein(a: str, b: str) -> int:
    m, n = len(a), len(b)
    dp = [[0]*(n+1) for _ in range(m+1)]
    for i in range(m+1):
        dp[i][0] = i
    for j in range(n+1):
        dp[0][j] = j
    for i in range(1, m+1):
        for j in range(1, n+1):
            cost = 0 if a[i-1] == b[j-1] else 1
            dp[i][j] = min(
                dp[i-1][j] + 1,
                dp[i][j-1] + 1,
                dp[i-1][j-1] + cost
            )
    return dp[m][n]

def cer_loss(pred: str, label: str) -> float:
    if len(label) == 0:
        return 0.0 if len(pred) == 0 else 1.0
    dist = levenshtein(pred, label)
    return dist / len(label)

# LOSS HERE
pred  = "OF. EUGENIO TIMENEE R. Ministro"
label = "Dr. Eugenio Jiménez R Ministro"
print("CER:", cer_loss(pred, label))

CER: 0.5333333333333333
